In [2]:
from datasets import load_dataset
import pandas as pd
dolly = load_dataset("databricks/databricks-dolly-15k")
dolly = pd.DataFrame(dolly)
alpaca = pd.read_json("../stanford_alpaca/alpaca_data.json")


In [6]:
proportions = {"alpaca": 0.7, "dolly": 0.3, "train": 0.6, "validation": 0.4}
total_size = len(alpaca) + len(dolly)
total_size

67013

In [7]:
train_size = int(proportions["train"] * total_size)
validation_size = int(proportions["validation"] * total_size)

In [10]:
import numpy as np
alpaca_train_indices = np.random.randint(0, len(alpaca), size=int(train_size * proportions["alpaca"]))
dolly_train_indices = np.random.randint(0, len(dolly), size=int(train_size * proportions["dolly"]))                          
extract_alpaca_train = alpaca.iloc[alpaca_train_indices]
extract_dolly_train = dolly.iloc[dolly_train_indices]

alpaca_remaining = alpaca.drop(index=alpaca_train_indices)
dolly_remaining = dolly.drop(index=dolly_train_indices)

In [11]:
alpaca_validation_indices = np.random.randint(0, len(alpaca_remaining), size=int(validation_size * proportions["alpaca"]))
dolly_validation_indices = np.random.randint(0, len(dolly_remaining), size=int(validation_size * proportions["dolly"]))

try:
    extract_alpaca_validation = alpaca.iloc[alpaca_validation_indices]
    extract_dolly_validation = dolly.iloc[dolly_validation_indices]
except IndexError as e:
    print("Pas assez long")

In [12]:
train = extract_alpaca_train.concat(extract_dolly_train, axis=1)
validation = extract_alpaca_validation.concat(extract_dolly_validation, axis=1)

TypeError: concat() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given